In [84]:
import torch
import pandas as pd
import numpy as np

df = pd.read_csv("data/bank-full.csv",sep=";")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
df.head()
print(device)
for i in range(torch.cuda.device_count()):
    print(f"GPU {i} 이름:", torch.cuda.get_device_name(i))

# from sklearn.preprocessing import LabelEncoder
# x = df.drop('y',axis=1).values
# le = LabelEncoder()
# y = le.fit_transform(df['y'])



cuda
GPU 0 이름: NVIDIA GeForce GTX 1660 Ti


## 범주형 변수 인코딩

In [85]:
from sklearn.preprocessing import LabelEncoder

list1 = ['job','marital','education','default','housing','loan','contact','month','poutcome','y']

for i in list1:
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i])



x = df.drop('y',axis=1).values
y = df['y'].values.reshape(-1,1)




## 스케일링 및 텐서화

In [86]:
from sklearn.preprocessing import StandardScaler

scaler1 = StandardScaler()

X_scaled = scaler1.fit_transform(x)


X_tensor = torch.tensor(X_scaled,dtype=torch.float32)
y_tensor = torch.tensor(y,dtype=torch.float32)

## 훈련 데이터 학습데이터 나누기

In [87]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_tensor,y_tensor,test_size=0.2)



In [88]:
print(y_train.unique())

tensor([0., 1.])


## 배치 설정

In [89]:
from torch.utils.data import DataLoader,TensorDataset

train_dataset = TensorDataset(X_train,y_train)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,num_workers=8)


## 모델 설정

In [ ]:
import torch.nn as nn

class binaryClf(nn.Module):

    def __init__(self,input_dim):

        super().__init__()
        self.linear1 = nn.Linear(input_dim,32)
        self.linear2 = nn.Linear(32,16)
        self.output = nn.Linear(16,1)
        self.relu = nn.ReLU()

    def forward(self,x):
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = (self.output(x))
        return x


model = binaryClf(input_dim = X_train.shape[-1]).to(device)


## 손실함수 설정

In [91]:
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

In [ ]:
nb_epoch = 3000

for epoch in range(nb_epoch):
    model.train()
    total_loss = 0

    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        pred = model(batch_x)
        loss = criterion(pred, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()


    # if epoch % 100 == 0:
    model.eval()
    with torch.no_grad():
        test_pred = model(X_test.to(device))
        test_loss = criterion(test_pred, y_test.to(device))
        acc = ((test_pred >= 0.5).float() == y_test.to(device)).float().mean()
        print(f"{epoch}/{nb_epoch} | Test Loss: {test_loss.item():.4f} | Test Acc: {acc.item() * 100:.2f}%")
        



0/3000 | Test Loss: 0.2282 | Test Acc: 89.42%
1/3000 | Test Loss: 0.2344 | Test Acc: 89.86%
2/3000 | Test Loss: 0.2271 | Test Acc: 89.68%
3/3000 | Test Loss: 0.2306 | Test Acc: 89.73%
4/3000 | Test Loss: 0.2277 | Test Acc: 89.77%
5/3000 | Test Loss: 0.2237 | Test Acc: 89.91%
6/3000 | Test Loss: 0.2258 | Test Acc: 89.59%
7/3000 | Test Loss: 0.2241 | Test Acc: 89.61%
8/3000 | Test Loss: 0.2301 | Test Acc: 89.51%
